## 0806-博客现目-标签云、浏览排行、站长推荐


## 标签云

添加标签云需要在models模型类中添加标签表

### 配置models.py

```python
class Label(models.Model):
    # 标签表
    name = models.CharField(max_length=32)

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = "标签表"
        verbose_name_plural = verbose_name
```

### 标签表注册到admin后台

```python
admin.site.register(Label)
```

### 配置全局函数Global

标签云在所有页面都显示，所以直接设置全局函数，使函数在所有页面中均显示。

1. 在视图函数`views.py`中创建Global函数
```python
# 全局变量：每一个模板都能使用的函数
# 定义全局变量，必须在settings.py中templates里添加过来
def Global(request):
    # 函数内容后续添加
    return locals()
```

2. 然后在`settings.py`文件中的`TEMPLATES`变量中将全局函数添加到模板页面，以供所有页面调用全局函数
```python
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [os.path.join(BASE_DIR, 'templates')],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
                'demo.views.Global',        # 添加全局变量
            ],
        },
    },
]
```

### 标签云与分类表配置到全局函数

配置完全局函数，就要对全局函数进行配置。博客中除标签云以外，分类表的显示也是所有页面都需要调用的，所以也把分类表的显示移到全局函数中
```python
def Global(request):
    all_label = Label.objects.all()
    # 获取所有标签
    all_category = Category.objects.all()
    # 获取所有分类
 
    return locals()
```

### 配置模板HTML

```html
<h3>标签云</h3>
      <ul>
          {% for label in all_label %}
          <li><a href="#">{{ label.name }}</a></li>
          {% endfor %}

      </ul>
```

### 生成迁移、执行迁移、启动服务

生成迁移
```python
python manage.py makemigrations
```

执行迁移
```python
python manage.py migrate
```

启动服务
```python
python manage.py runserver
```



### 后台添加标签数据查看效果

----------------------------

## 浏览排行和站长推荐

> 浏览排行主要根据浏览量数据进行排序，并指定输出前几个数据  
> 站长推荐主要根据是否推荐选项按照发布时间进行排序，并指定数据前几个数据  



### 配置视图函数views

在全局函数中配置

```python

def Global(request):
    all_label = Label.objects.all()
    # 获取所有标签
    all_category = Category.objects.all()
    # 获取所有分类
    all_article_views = Article.objects.all().order_by('-page_views')[0:4]
    # 获取浏览排行前四的文章
    all_article_recommend = Article.objects.filter(recommend=True).order_by('-publish_time')[:4]
    # 获取所有推荐的文章,发布时间倒叙排序的前四个
    return locals()
```

### 配置模板HTML

```html
 <div class="ms-top">
        <ul class="hd" id="tab">
          <li class="cur"><a href="/">浏览排行</a></li>
          <li><a href="/">评论排行</a></li>
          <li><a href="/">站长推荐</a></li>
        </ul>
 </div>
 <div class="ms-main" id="ms-main">
        <div style="display: block;" class="bd bd-news" >
          <ul>
              {% for article in all_article_views %}
              <li><a href="/article/{{ article.id }}/" target="_blank">{{ article.title }}</a></li>
              {% endfor %}

             </ul>
        </div>
        <div  class="bd bd-news">
          <ul>

            <li><a href="/article/{{ article.id }}/" target="_blank"></a></li>
          </ul>
        </div>
        <div class="bd bd-news">
          <ul>
             {% for article in all_article_recommend %}
              <li><a href="/article/{{ article.id }}/" target="_blank">{{ article.title}}</a></li>
              {% endfor %}
          </ul>
        </div>
 </div>

```

### 启动服务

```python
python manage.py runserver
```